In [ ]:
# library imports
import argparse
import os
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
import pandas as pd
import cv2

In [ ]:
root = "data"

ct_root = os.path.join(root, 'image')
np_root = os.path.join(root, 'numpy')
resampled_root = os.path.join(root, 'resampled')

# create numpy and resampled directory
if not os.path.isdir(np_root):
    os.makedirs(np_root)
if not os.path.isdir(resampled_root):
    os.makedirs(resampled_root)
ct_data = os.listdir(ct_root)
print(ct_data)
patient_number = []
for i in ct_data:
    if 'roi' not in i:
        C, nii, gz = i.split('.')
        try:
            c, num = C.split('-')
        except:
            num = C
        patient_number.append(str(int(num)))
print(patient_number)
# Resampling Method

In [ ]:

def CT_resampling(ct_root, ct_data, resampled_root):
    for data in ct_data:
        if 'roi' not in data:
            C, nii, gz = data.split('.')
            try:
                c, num = C.split('-')
            except:
                num = C
            num = str(int(num))

            # getting numpy array from nii.gz
            img_root = os.path.join(ct_root, data)
            img = sitk.ReadImage(img_root)
            img_np = sitk.GetArrayFromImage(img)
            img_min = int(img_np.min())
            size = img.GetSize()

            # 
            reference_np = np.zeros((size[2], size[1], size[0]))
            reference_img = sitk.GetImageFromArray(reference_np)
            reference_img.CopyInformation(img)
            reference_img.SetSpacing([1.0, 1.0, 1.0])

            # 
            resampler = sitk.ResampleImageFilter()
            resampler.SetReferenceImage(reference_img)
            resampler.SetInterpolator(sitk.sitkBSplineResamplerOrder3)
            resampler.SetDefaultPixelValue(img_min)

            # 
            orig_size = np.array(img.GetSize(), dtype=int)
            orig_spacing = img.GetSpacing()
            new_size = orig_size*(orig_spacing)
            new_size = np.ceil(new_size).astype(int)
            new_size = [int(s) for s in new_size]
            resampler.SetSize(new_size)

            # Resampling
            print(f"Resampling {num} from {orig_spacing} to {[1.0, 1.0, 1.0]}")
            new_img = resampler.Execute(img)
            save_pth = os.path.join(resampled_root, num)
            if not os.path.isdir(save_pth):
                os.makedirs(save_pth)

            resample_path = os.path.join(save_pth, data)
            sitk.WriteImage(new_img, resample_path)

def ROI_resampling(ct_root, ct_data, resampled_root):
    for data in ct_data:
        if 'roi' in data:
            C, roi, nii, gz = data.split('.')
            try:
                c, num = C.split('-')
            except:
                num = C
            num = str(int(num))

            roi_root = os.path.join(ct_root, data)
            roi = sitk.ReadImage(roi_root)
            roi_np = sitk.GetArrayFromImage(roi)
            roi_min = int(roi_np.min())
            size = roi.GetSize()

            reference_np = np.zeros((size[2], size[1], size[0]))
            reference_img = sitk.GetImageFromArray(reference_np)
            reference_img.CopyInformation(roi)
            reference_img.SetSpacing([1.0, 1.0, 1.0])

            resampler = sitk.ResampleImageFilter()
            resampler.SetReferenceImage(reference_img)
            resampler.SetInterpolator(sitk.sitkNearestNeighbor)
            resampler.SetDefaultPixelValue(roi_min)

            orig_size = np.array(roi.GetSize(), dtype=int)
            orig_spacing = roi.GetSpacing()
            new_size = orig_size*(orig_spacing)
            new_size = np.ceil(new_size).astype(int)
            new_size = [int(s) for s in new_size]
            resampler.SetSize(new_size)

            # Resampling roi
            print(f"Resampling {num} from {orig_spacing} to {[1.0, 1.0, 1.0]}")
            new_roi = resampler.Execute(roi)

            save_pth = os.path.join(resampled_root, num)
            if not os.path.isdir(save_pth):
                os.makedirs(save_pth)

            resample_roi_path = os.path.join(save_pth, data)
            sitk.WriteImage(new_roi, resample_roi_path)

CT_resampling(ct_root, ct_data, resampled_root)
# Resample ROI
ROI_resampling(ct_root, ct_data, resampled_root)

In [ ]:

def image_slice_check(ct_root, ct_data):
    thick = {}
    z_thick = {}
    for data in ct_data:
        if 'roi' in data :
            C, roi, nii, gz = data.split('.')
            try:
                c, num = C.split('-')
            except:
                num = C
            num = str(int(num))

            roi_origin_root = os.path.join(ct_root, data)
            roi_origin = sitk.ReadImage(roi_origin_root)
            spacing = roi_origin.GetSpacing()
            thick[num] = spacing
            z_thick[num] = roi_origin.GetSpacing()[2]

    thickness = [value for key, value in z_thick.items()]
    thickness_set = set(thickness)
    thick_count = {}
    for i in thickness_set:
        thick_count[i] = 0
        for j in thickness :
            if i == j :
                thick_count[i] += 1
    return thick_count

def ROI_max_slice(resampled_root):
    per_patient_max_slice = np.array([])
    per_patient_max_slice_dict = {}
    per_patient_total_max_slice_dict = {}
    patient_no = os.listdir(resampled_root)
    for no in patient_no:
        patient_root = os.path.join(resampled_root, no)
        patient_data = os.listdir(patient_root)
        for data in patient_data:
            if 'roi' in data:
                roi_root = os.path.join(patient_root, data)
                roi = sitk.ReadImage(roi_root)
                roi_np = sitk.GetArrayFromImage(roi)
                roi_min = int(roi_np.min())

                per_slice_roi_pixel_count = np.array([])
                for i in range(len(roi_np)):
                    roi_z = roi_np[i]
                    roi_count = len(np.where(roi_z != roi_min)[0])
                    per_slice_roi_pixel_count = np.append(per_slice_roi_pixel_count, roi_count)

                max_slice = np.max(per_slice_roi_pixel_count)
                z_slice_no = np.where(per_slice_roi_pixel_count == max_slice)[0]
                per_patient_max_slice = np.append(per_patient_max_slice, z_slice_no)
                per_patient_total_max_slice_dict[str(no)] = z_slice_no
                per_patient_max_slice_dict[str(no)] = z_slice_no[0]


    return per_patient_max_slice, per_patient_max_slice_dict, per_patient_total_max_slice_dict

In [ ]:

def ROI_column_raw_check(resampled_root, per_patient_max_slice_dict):
    row_col_check_dict = {}
    row_col_check_np = np.array([])
    row_col_check_np = row_col_check_np.reshape(0, 4)
    patient_no = os.listdir(resampled_root)
    for no in patient_no:
        patient_root = os.path.join(resampled_root, no)
        patient_data = os.listdir(patient_root)
        col_check = np.array([])
        row_check = np.array([])

        for data in patient_data:
            if 'roi' in data:
                roi_root = os.path.join(patient_root, data)
                roi_img = sitk.ReadImage(roi_root)
                roi_np = sitk.GetArrayFromImage(roi_img)
                roi_min = int(roi_np.min())

                max_slice_no = per_patient_max_slice_dict[str(no)]
                max_slice = roi_np[max_slice_no]
                for i in range(max_slice.shape[0]):
                    roi_count = np.where(max_slice[i] != roi_min)[0]
                    col_check = np.append(col_check, roi_count)
                col_check_unique = np.unique(col_check)

                for j in range(max_slice.shape[1]):
                    roi_count = np.where(max_slice[:, j] != roi_min)[0]
                    row_check = np.append(row_check, roi_count)
                row_check_unique = np.unique(row_check)

                row_min = int(row_check_unique.min())
                row_max = int(row_check_unique.max())
                col_min = int(col_check_unique.min())
                col_max = int(col_check_unique.max())

                idx = [row_min, row_max, col_min, col_max]
                idx_np = np.array(idx)
                idx_np = idx_np.reshape(1, 4)

        row_col_check_dict[no] = idx
        row_col_check_np = np.append(row_col_check_np, idx_np, axis = 0)
    return row_col_check_dict, row_col_check_np

def ROI_center(row_col_check_dict):
    roi_size_per_patient_dict = {}
    roi_size_dict = {}
    roi_size_per_patient_np = np.array([])
    roi_size_per_patient_np = roi_size_per_patient_np.reshape(0, 2)
    roi_center_per_patient_dict = {}
    roi_center_per_patient_np = np.array([])
    roi_center_per_patient_np = roi_center_per_patient_np.reshape(0, 2)

    for key in row_col_check_dict:
        length = row_col_check_dict[key][1] - row_col_check_dict[key][0]
        mid_length = int((row_col_check_dict[key][1] + row_col_check_dict[key][0]) / 2)
        width = row_col_check_dict[key][3] - row_col_check_dict[key][2]
        mid_width = int((row_col_check_dict[key][3] + row_col_check_dict[key][2]) / 2)
        size = [length, width]
        center = [mid_length, mid_width]
        size_np = np.array(size)
        size_np = size_np.reshape(1, 2)
        center_np = np.array(center)
        center_np = center_np.reshape(1, 2)
        roi_size_per_patient_dict[key] = size
        roi_size_dict[key] = max(size)
        roi_size_per_patient_np = np.append(roi_size_per_patient_np, size_np, axis = 0)
        roi_center_per_patient_dict[key] = center
        roi_center_per_patient_np = np.append(roi_center_per_patient_np, center_np, axis = 0)
    return roi_center_per_patient_dict, roi_size_dict

In [ ]:
thick_count = image_slice_check(ct_root, ct_data)
per_patient_max_slice, per_patient_max_slice_dict, per_patient_total_max_slice_dict = ROI_max_slice(resampled_root)
row_col_check_dict, row_col_check_np = ROI_column_raw_check(resampled_root, per_patient_max_slice_dict)
roi_center_per_patient_dict, roi_size_dict = ROI_center(row_col_check_dict)
roi_max_size = max([value for (key, value) in roi_size_dict.items()])
cut_size = max(64, roi_max_size / 2)
print(roi_max_size, cut_size)
axial_slice_three(resampled_root, np_root, per_patient_max_slice_dict, roi_center_per_patient_dict, cut_size = 64)
min_intensity(np_root)
normalization(np_root)
